<a href="https://colab.research.google.com/github/MahdiTheGreat/Intro-to-language-modeling/blob/main/intro_to_language_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/MahdiTheGreat/Intro-to-language-modeling.git
%cd Intro-to-language-modeling

Cloning into 'Intro-to-language-modeling'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 34 (delta 17), reused 2 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (34/34), 28.30 MiB | 24.17 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/content/Intro-to-language-modeling/Intro-to-language-modeling


In [ ]:
import sklearn

In [ ]:
!pip install ipdb
!pip install -U spacy
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.1/29.1 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 92.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: blis
    Found existing installation: blis 0.7.11
    Uninstalling blis-0.7.11:
      Successfully uninstalled blis-0.7.11
  Attempting uninstall: thinc
    Found existing installation: thinc 8.2.5
    Uninstalling thinc-8.2.5:
      Successfully uninstalled thinc-8.2.5
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.5
    Uninst

In [ ]:
import spacy
import torch
import matplotlib.pyplot as plt
import ipdb
import numpy as np
import random
import pandas as pd


In [ ]:
%pdb on

Automatic pdb calling has been turned ON


In [ ]:
# Set random seed for reproducibility
def set_seed(seed=2024):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

set_seed(1998)

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu'))
print(f'Using device: {device}')

Using device: cpu


In [ ]:
dataset='lmdemo'
zip_file = f"{dataset}.zip"
!unzip -q $zip_file
!rm $zip_file

In [ ]:
text=open(f'{dataset}/train.txt','r',encoding='utf-8').read()

In [32]:
example_filepath="example.txt"

In [38]:
import spacy
from collections import Counter
import json


def get_token_counter(filepath,nlp,encoding):

    buffer = ""  # Buffer to store partial sentences between lines
    token_counter = Counter()

    def sent_processor(sent,complete=True):
     tokens=[]
     if complete: tokens.append(nlp("BEGINNING")[0])  # Add "BEGINNING" at the start of each sentence
     tokens.extend([token for token in sent])  # Add sentence tokens
     if complete: tokens.append(nlp("END")[0])  # Add "END" at the end of each sentence
     for token in tokens:
      if not token.is_space:
       token_counter[token.text.lower()] += 1


    with open(filepath, 'r') as file:


        for line in file:
            # Add line to buffer and process with spaCy
            buffer += " " + line.strip()
            doc = nlp(buffer)
            # Extract complete sentences
            sentences = list(doc.sents)
            for i, sent in enumerate(sentences):
                # If it's not the last sentence, we print it as it's complete
                if i < len(sentences) - 1:
                    print(sent)
                    sent_processor(sent)
                else:
                    # If it's the last sentence, store it in the buffer in case it's incomplete
                    buffer = sent.text
                    # Process sentences and identify complete sentences
            for sent in doc.sents:
                if sent.end_char < len(buffer):
                    print(sent)
                    sent_processor(sent)

        # Process any remaining content in the buffer
        doc = nlp(buffer)
        for sent in doc.sents:
         print(sent)
         sent_processor(sent)

    return token_counter

# Load spaCy model for tokenization
nlp = spacy.load("en_core_web_sm")

class VocabularyBuilder:
    def __init__(self, max_voc_size=None):
        self.max_voc_size = max_voc_size
        self.str_to_int = {}
        self.int_to_str = {}
        self.special_tokens = ["BEGINNING", "END", "UNKNOWN"]
        self.token_counter = None


    def build_vocabulary(self, filepath, nlp,token_counter_savepath=None,token_counter_loadpath=None,encoding="utf-8"):

        # Tokenize text and count tokens
        if token_counter_loadpath is not None:
         with open(token_counter_loadpath, "r") as file:
            self.token_counter = Counter(json.load(file))
        else:
         self.token_counter =get_token_counter(filepath=filepath,nlp=nlp,encoding=encoding)

        # Start vocabulary with special tokens
        for idx, token in enumerate(self.special_tokens):
            self.str_to_int[token] = idx
            self.int_to_str[idx] = token

        # Select the most common tokens, considering max_voc_size - len(special_tokens)
        if self.max_voc_size is None:
            max_words = len(self.token_counter) - len(self.special_tokens)
            self.max_voc_size = max_words + len(self.special_tokens)
        else:
         max_words = self.max_voc_size - len(self.special_tokens)
        most_common_tokens = self.token_counter.most_common(max_words)

        for idx, (token, _) in enumerate(most_common_tokens, start=len(self.special_tokens)):
            self.str_to_int[token] = idx
            self.int_to_str[idx] = token

        # Save to a JSON file
        if token_counter_savepath is not None:
         with open(token_counter_savepath, "w") as file:
             json.dump(self.token_counter, file)


    def get_token_id(self, token):
        # Return the integer ID for a given token
        return self.str_to_int.get(token.lower(), self.str_to_int["UNKNOWN"])

    def get_token_str(self, token_id):
        # Return the original token string for a given integer ID
        return self.int_to_str.get(token_id, "UNKNOWN")

    def add_special_tokens_to_text(self, text):
        """
        Tokenizes the text by sentence and adds special 'BEGINNING' and 'END' tokens
        around each sentence.

        Parameters:
        - text (str): The input text.

        Returns:
        - List[str]: A list of tokens with special 'BEGINNING' and 'END' tokens added.
        """


    def sanity_check(self):
        # Check vocabulary size
        assert len(self.str_to_int) <= self.max_voc_size, "Vocabulary size exceeds max_voc_size."

        # Check special tokens exist and are unique
        for token in self.special_tokens:
            assert token in self.str_to_int, f"Missing special token: {token}"

        # Check if highly frequent words are included and rare ones are not
        common_words = ["the", "and"]
        rare_words = ["cuboidal", "epiglottis"]

        for word in common_words:
            assert word in self.str_to_int, f"Common word '{word}' not in vocabulary."

        for word in rare_words:
            assert word not in self.str_to_int, f"Rare word '{word}' should not be in vocabulary."

        # Check that mapping back and forth works for a test word
        test_word = "The"
        token_id = self.get_token_id(test_word)
        assert self.get_token_str(token_id) == test_word.lower(), "Round-trip token mapping failed."

        print("Sanity check passed!")

token_counter_filepath="token_counter.json"
vocab_builder = VocabularyBuilder()
vocab_builder.build_vocabulary(filepath=example_filepath, nlp=nlp,token_counter_savepath=token_counter_filepath)

# Example mappings
print("str_to_int:", vocab_builder.str_to_int)
print("int_to_str:", vocab_builder.int_to_str)
print("vocabulary size: ",len(vocab_builder.token_counter))

# Convert a token to integer ID and back to string
token_id = vocab_builder.get_token_id("example")
print("Token ID for 'example':", token_id)
print("Original token from ID:", vocab_builder.get_token_str(token_id))

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.8.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


 Anatomy Anatomy (Greek anatomē, “dissection”) is the branch of biology concerned with the study of the structure of organisms and their parts.  
Anatomy is a branch of natural science dealing with the structural organization of living things.  
It is an old science, having its beginnings in prehistoric times.  
Anatomy is inherently tied to embryology, comparative anatomy, evolutionary biology, and phylogeny, as these are the processes by which anatomy is generated over immediate (embryology) and long (evolution) timescales.  
Human anatomy is one of the basic essential sciences of medicine.
The discipline of anatomy is divided into macroscopic and microscopic anatomy.  
Macroscopic anatomy, or gross anatomy, is the examination of an animal's body parts using unaided eyesight.  
Gross anatomy also includes the branch of superficial anatomy.  
Human anatomy is one of the basic essential sciences of medicine.
The discipline of anatomy is divided into macroscopic and microscopic anatomy.

In [39]:
token_counter_filepath="token_counter.json"
vocab_builder = VocabularyBuilder()
vocab_builder.build_vocabulary(filepath=example_filepath, nlp=nlp,token_counter_loadpath=token_counter_filepath)

# Example mappings
print("str_to_int:", vocab_builder.str_to_int)
print("int_to_str:", vocab_builder.int_to_str)
print("vocabulary size: ",len(vocab_builder.token_counter))


# Convert a token to integer ID and back to string
token_id = vocab_builder.get_token_id("example")
print("Token ID for 'example':", token_id)
print("Original token from ID:", vocab_builder.get_token_str(token_id))

str_to_int: {'BEGINNING': 0, 'END': 1, 'UNKNOWN': 2, 'anatomy': 3, 'of': 4, 'the': 5, 'beginning': 6, '.': 7, 'end': 8, ',': 9, 'is': 10, 'and': 11, '(': 12, ')': 13, 'branch': 14, 'study': 15, 'in': 16, 'macroscopic': 17, 'microscopic': 18, 'biology': 19, 'with': 20, 'parts': 21, 'science': 22, 'an': 23, 'embryology': 24, 'as': 25, 'human': 26, 'one': 27, 'basic': 28, 'essential': 29, 'sciences': 30, 'medicine': 31, 'discipline': 32, 'divided': 33, 'into': 34, 'gross': 35, 'also': 36, 'greek': 37, 'anatomē': 38, '“': 39, 'dissection': 40, '”': 41, 'concerned': 42, 'structure': 43, 'organisms': 44, 'their': 45, 'a': 46, 'natural': 47, 'dealing': 48, 'structural': 49, 'organization': 50, 'living': 51, 'things': 52, 'it': 53, 'old': 54, 'having': 55, 'its': 56, 'beginnings': 57, 'prehistoric': 58, 'times': 59, 'inherently': 60, 'tied': 61, 'to': 62, 'comparative': 63, 'evolutionary': 64, 'phylogeny': 65, 'these': 66, 'are': 67, 'processes': 68, 'by': 69, 'which': 70, 'generated': 71, 'ov

In [40]:
# Perform sanity check
vocab_builder.sanity_check()

Sanity check passed!


In [71]:
import csv
class TrainingDataPreparer:
    def __init__(self, vocab_builder, nlp, context_window_size=3, chunk_size=1024):
        self.vocab_builder = vocab_builder
        self.context_window_size = context_window_size
        self.chunk_size = chunk_size
        self.nlp = nlp

    def encode_token(self, token):
        token_id = self.vocab_builder.get_token_id(token.text)
        return token_id if token_id != self.vocab_builder.get_token_id("UNKNOWN") else None

    def prepare_training_data(self, input_file, output_file):
        with open(input_file, "r") as infile, open(output_file, "w", newline="") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([f"Token_{i+1}" for i in range(self.context_window_size)] + ["Target"])

            # Initialize the beginning padding tokens
            padded_tokens = [self.vocab_builder.get_token_id("BEGINNING")] * self.context_window_size
            first_chunk = True

            while True:
                chunk = infile.read(self.chunk_size)
                if not chunk:
                    break

                # Tokenize chunk into sentences
                doc = self.nlp(chunk)
                sentences = list(doc.sents)

                for sentence in sentences:
                    # Process sentence and convert to token IDs, skipping unknowns and spaces
                    sentence_token_ids = [
                        self.encode_token(token) for token in sentence if self.encode_token(token) is not None
                    ]

                    if first_chunk and sentence_token_ids:
                        padded_tokens += sentence_token_ids
                        first_chunk = False
                    else:
                        # Add only the sentence tokens from subsequent sentences
                        padded_tokens.extend(sentence_token_ids)

                    # Add END token at the end of each sentence
                    padded_tokens.append(self.vocab_builder.get_token_id("END"))

                    # Generate context-target sequences
                    for i in range(len(padded_tokens) - self.context_window_size):
                        context = padded_tokens[i:i + self.context_window_size]
                        target = padded_tokens[i + self.context_window_size]
                        writer.writerow(context + [target])

                # Retain only the last context window tokens for the next chunk
                padded_tokens = padded_tokens[-self.context_window_size:]

        print("Training data preparation complete.")

    def print_csv_as_words(self, csv_file):
           """
           Reads a CSV file with token IDs, decodes them to words, and prints each sequence.
           """
           with open(csv_file, "r") as file:
               reader = csv.reader(file)
               headers = next(reader)  # Skip the header

               for row in reader:
                   context_ids = row[:-1]  # All columns except the last one are context
                   target_id = row[-1]  # Last column is the target

                   # Convert token IDs to words
                   context_words = [self.vocab_builder.get_token_str(int(token_id)) for token_id in context_ids]
                   target_word = self.vocab_builder.get_token_str(int(target_id))

                   # Print context and target as words
                   print("Context:", context_words, "-> Target:", target_word)


data_preparer = TrainingDataPreparer(vocab_builder=vocab_builder,nlp=nlp, context_window_size=3)

input_file = "example.txt"
output_file = "training_sequences.csv"

# Prepare training data
data_preparer.prepare_training_data(input_file, output_file)


Training data preparation complete.


In [72]:
training_data=pd.read_csv("training_sequences.csv")
print(training_data)

     Token_1  Token_2  Token_3  Target
0          6        6        6       3
1          6        6        3       3
2          6        3        3      12
3          3        3       12      37
4          3       12       37      38
..       ...      ...      ...     ...
954       11       36       16       5
955       36       16        5      15
956       16        5       15       4
957        5       15        4       7
958       15        4        7       8

[959 rows x 4 columns]


In [75]:
data_preparer.print_csv_as_words("training_sequences.csv")

Context: ['beginning', 'beginning', 'beginning'] -> Target: anatomy
Context: ['beginning', 'beginning', 'anatomy'] -> Target: anatomy
Context: ['beginning', 'anatomy', 'anatomy'] -> Target: (
Context: ['anatomy', 'anatomy', '('] -> Target: greek
Context: ['anatomy', '(', 'greek'] -> Target: anatomē
Context: ['(', 'greek', 'anatomē'] -> Target: ,
Context: ['greek', 'anatomē', ','] -> Target: “
Context: ['anatomē', ',', '“'] -> Target: dissection
Context: [',', '“', 'dissection'] -> Target: ”
Context: ['“', 'dissection', '”'] -> Target: )
Context: ['dissection', '”', ')'] -> Target: is
Context: ['”', ')', 'is'] -> Target: the
Context: [')', 'is', 'the'] -> Target: branch
Context: ['is', 'the', 'branch'] -> Target: of
Context: ['the', 'branch', 'of'] -> Target: biology
Context: ['branch', 'of', 'biology'] -> Target: concerned
Context: ['of', 'biology', 'concerned'] -> Target: with
Context: ['biology', 'concerned', 'with'] -> Target: the
Context: ['concerned', 'with', 'the'] -> Target: stu

In [ ]:
## Creating a tensor dataset ##
from torch.utils.data import DataLoader, TensorDataset
def TorchDataLoader(training_sequences, batch_size):
  context_words = [item[0] for item in training_sequences]  # List of [context]
  target_words = [item[1] for item in training_sequences]   # List of target words

  # Convert lists to tensors
  context_tensor = torch.tensor(context_words, dtype=torch.long)  # Shape: (num_samples, 3)
  target_tensor = torch.tensor(target_words, dtype=torch.long)    # Shape: (num_samples,)

  # Create a TensorDataset
  dataset = TensorDataset(context_tensor, target_tensor)

  # Create a DataLoader for batching
  batch_size = 4
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

  # Iterate over the DataLoader
  for batch_context, batch_target in dataloader:
      print("Batch context:", batch_context)
      print("Batch target:", batch_target)
      # You can now use batch_context and batch_target for model training

  return dataloader

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim


class SimpleANN(nn.Module):

    def __init__(self,layer_sizes,activation=nn.ReLU,last_layer_activation=nn.ReLU,dropout=0):

        super(SimpleANN, self).__init__()
        self.layers = nn.ModuleList()

        for i in range(len(layer_sizes)-2):
          self.layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
          self.layers.append(nn.Dropout(dropout))
          self.layers.append(activation())

        self.layers.append(nn.Linear(layer_sizes[-2], layer_sizes[-1]))
        if last_layer_activation is not None:
         self.layers.append(nn.Dropout(dropout))
         self.layers.append(last_layer_activation())

    def forward(self, x):
        x = x.view(-1, np.prod(x.shape[1:])) # Flatten the input
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
from copy import deepcopy
def dense_arch_builder(input_size,scale_factor=0,hidden_layers_num=0,repeat=0,output_size=1):
  layer_sizes=[input_size]

  if scale_factor!=0:

   if scale_factor>1:
    for i in range(hidden_layers_num):
     layer_sizes.append(layer_sizes[-1]*scale_factor)
    while layer_sizes[-1]<output_size:
     layer_sizes.append(layer_sizes[-1]*scale_factor)

   elif scale_factor==1:
     for i in range(2,hidden_layers_num+2):
      layer_sizes.append(layer_sizes[0]*i)
     i+=1
     while layer_sizes[-1]<output_size:
      layer_sizes.append(layer_sizes[0]*i)
      i+=1

   mirrored_layer_sizes=deepcopy(layer_sizes)
   mirrored_layer_sizes.reverse()
   mirrored_layer_sizes=mirrored_layer_sizes[1:-1]

   for i in range(repeat):
    layer_sizes.append(layer_sizes[-1])

   if output_size>0:
    layer_sizes+=mirrored_layer_sizes
    downscale_factor=scale_factor if scale_factor>1 else 2

    while layer_sizes[-1]!=output_size:
     if layer_sizes[-1]//downscale_factor>=output_size:
      layer_sizes.append(layer_sizes[-1]//downscale_factor)
     else:
      layer_sizes.append(output_size)

  else:
    downscale_factor = (input_size / output_size) ** (1 / hidden_layers_num)
    for i in range(hidden_layers_num):
        layer_sizes.append(int(input_size / (downscale_factor ** i)))
    layer_sizes.append(output_size)

  return layer_sizes


In [ ]:
# Helper function to plot the training metrics

def plot_training_metrics(train_acc, val_acc, train_loss, title, save_path):
    # Ensure that all input lists have the same length
    assert len(train_acc) == len(val_acc) == len(train_loss), "All input histories must have the same length."

    epochs = range(1, len(train_acc) + 1)

    # Create the metrics DataFrame
    df_metrics = pd.DataFrame({
        'Epoch': epochs,
        'Training Accuracy (%)': train_acc,
        'Validation Accuracy (%)': val_acc,
        'Training Loss': train_loss
    })

    # Initialize the plot
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # Plot Training and Validation Accuracy on ax1
    color = 'tab:blue'
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy (%)', color=color)
    ax1.plot(df_metrics['Epoch'], df_metrics['Training Accuracy (%)'], label='Train Acc', color='tab:blue')
    ax1.plot(df_metrics['Epoch'], df_metrics['Validation Accuracy (%)'], label='Val Acc', color='tab:cyan')
    ax1.tick_params(axis='y', labelcolor=color)

    # Create a second y-axis for Training Loss
    ax2 = ax1.twinx()
    color = 'tab:red'
    ax2.set_ylabel('Loss', color=color)
    ax2.plot(df_metrics['Epoch'], df_metrics['Training Loss'], label='Train Loss', color='tab:red')
    ax2.tick_params(axis='y', labelcolor=color)

    # Combine legends from both axes
    lines_1, labels_1 = ax1.get_legend_handles_labels()
    lines_2, labels_2 = ax2.get_legend_handles_labels()
    ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper left')

    # Set plot title and layout
    plt.title(title)
    plt.tight_layout()

    # Save and display the plot
    plt.savefig(save_path)
    plt.show()